In [4]:
#!pip install pycaret

## Import Libraries

In [6]:
import pandas as pd
from pycaret.classification import *

## Import Data

In [7]:
train = pd.read_csv("training.csv")
test = pd.read_csv("test.csv")

In [8]:
train.head()

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020


In [9]:
train['IsBadBuy'].value_counts()

0    64007
1     8976
Name: IsBadBuy, dtype: int64

## Setup PyCaret with Target Variable

In [ ]:
exp_clf = setup(train, target = 'IsBadBuy')

IntProgress(value=0, description='Processing: ', max=13)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:36:48
Status,. . . . . . . . . . . . . . . . . .,Fitting Model on 40% sample
ETC,. . . . . . . . . . . . . . . . . .,2.0 Minutes Remaining


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
RefId,Numeric
IsBadBuy,Label
PurchDate,Date
Auction,Categorical
VehYear,Categorical
VehicleAge,Categorical
Make,Categorical
Model,Categorical
Trim,Categorical
SubModel,Categorical


In [10]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Extreme Gradient Boosting,0.9004,0.7609,0.2297,0.8541,0.3614,0.3264,0.4112,11.6315
1,CatBoost Classifier,0.8995,0.7619,0.2366,0.8172,0.3660,0.3291,0.4054,15.7022
2,Light Gradient Boosting Machine,0.8994,0.7526,0.2409,0.8063,0.3705,0.3328,0.4060,1.4544
3,Gradient Boosting Classifier,0.8992,0.7553,0.2334,0.8154,0.3625,0.3256,0.4023,38.1254
4,Extra Trees Classifier,0.8974,0.7335,0.2393,0.7679,0.3641,0.3247,0.3914,5.2762
5,Ada Boost Classifier,0.8934,0.7452,0.2239,0.7129,0.3404,0.2993,0.3598,18.3566
6,Random Forest Classifier,0.8923,0.6951,0.1889,0.7517,0.3008,0.2644,0.3406,0.5674
7,Ridge Classifier,0.8919,0.0000,0.2594,0.6523,0.3705,0.3233,0.3651,1.8079
8,Linear Discriminant Analysis,0.8818,0.7187,0.2960,0.5353,0.3809,0.3216,0.3390,11.6942
9,Logistic Regression,0.8770,0.6317,0.0000,0.0000,0.0000,0.0000,0.0000,3.0134


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=4383,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=0)

In [12]:
xgb = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9008,0.7843,0.2275,0.8776,0.3613,0.3272,0.4169
1,0.8995,0.7298,0.2063,0.9070,0.3362,0.3044,0.4049
2,0.9022,0.7965,0.2275,0.9149,0.3644,0.3316,0.4282
3,0.8950,0.7561,0.1905,0.8182,0.3090,0.2753,0.3633
4,0.9002,0.7761,0.2540,0.8000,0.3855,0.3467,0.4154
5,0.9014,0.7589,0.2394,0.8491,0.3734,0.3377,0.4191
6,0.8995,0.7279,0.2340,0.8148,0.3636,0.3268,0.4032
7,0.9047,0.7484,0.2660,0.8621,0.4065,0.3701,0.4470
8,0.9040,0.7751,0.2447,0.9020,0.3849,0.3509,0.4407
9,0.8962,0.7553,0.2074,0.7959,0.3291,0.2933,0.3730


In [ ]:
#xgb = load_model('xgb_Basic')

In [14]:
# AUC plot
plot_model(xgb, plot = 'auc')

In [20]:
test_pred = predict_model(xgb, data = test, probability_threshold = 0.5)

In [15]:
optimize_threshold(xgb, true_negative = 10, false_negative = -100)

NameError: name 'y' is not defined

In [25]:
test_pred['IsBadBuy'] = test_pred['Score'].apply(lambda x: 1 if x > 0.5 else 0)

In [28]:
test_pred[['RefId','IsBadBuy']].to_csv('PyCaret_xgb_Output.csv', index = False)

In [22]:
save_model(xgb, 'xgb_Basic', verbose=True)

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=4383,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=0), model_name=xgb_Basic, verbose=True)
INFO:logs:Appending prep pipeline
INFO:logs:xgb_Basic.pkl saved in current working directory
INFO:logs:[Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      ml_usecase='classification',
                                      numerical_features=[], targ

Transformation Pipeline and Model Succesfully Saved
